Title: Discovering Continuations with Typed Holes
Date: 2017-05-20
Status: draft
Category: programming

I've been trying to wrap my head around continuations for a while. I was reading [David Luposchainsky's excellent article](https://github.com/quchen/articles/blob/master/cont_monad.md) on them and playing with his definitions in an IHaskell notebook when I found that typed holes are an excellent hammer to attack this particular nail with.

If you haven't encountered them before, [typed holes](https://wiki.haskell.org/GHC/Typed_holes) are a feature where you put one or more variables starting with `_` on the right hand side of a definition to get GHC to tell you the type of the value that fits in that hole, and you can narrow the hole as necessary to get the type of a subexpression until you have enough information to complete the definition. I like to think of this as a way of collaboratively filling in a definition with the compiler, instead of my usual approach which is to write a definition, listen carefully to GHC's complaints, and amend my definition accordingly. Typed holes are fully supported by GHCi and the full suite of editor integrations, but I personally find the edit/reload/squint cycle more tedious than coming up with the definition in IHaskell and then moving it to a module and adding type signatures after I'm satisfied that it works.

I think his type definition makes an excellent starting point:

In [1]:
newtype Cont r a = Cont { (>>-) :: (a -> r) -> r }

This defines a type `Cont` with an infix constructor `>>-` (that looks suspiciously similar to `>>=`) that takes a function from `a` to `r` and provides an `r`. One intuition for what this means is that a value of this type knows about an `a` but for whatever reason refuses to be upfront about it and demands to know what you're going to do with it and then does it for you, providing you with a final result `r`. Another intuition is that this is a generalisation of callbacks: a value of this type expects a callback to utilise the `a`. Anyway, on to my favourite part of working with mysterious data types: defining `Functor`, `Applicative`, and `Monad` instances for them! If you've done this before, you'll know that these typeclasses have certain laws that their instances are meant to obey, and it turns out that this type is polymorphic enough that we can just follow the typed holes and the resulting definitions will be lawful. You don't have to take my word for it and should verify this for yourself, but I won't be discussing the laws here. Let's begin!

In [2]:
instance Functor (Cont r) where
    fmap f cont = _

<interactive>:2:19: error:
    • Found hole: _ :: Cont r b
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
             ‘b’ is a rigid type variable bound by
               the type signature for:
                 fmap :: forall a b. (a -> b) -> Cont r a -> Cont r b
               at <interactive>:2:5
    • In the expression: _
      In an equation for ‘fmap’: fmap f cont = _
      In the instance declaration for ‘Functor (Cont r)’
    • Relevant bindings include
        cont :: Cont r a (bound at <interactive>:2:12)
        f :: a -> b (bound at <interactive>:2:10)
        fmap :: (a -> b) -> Cont r a -> Cont r b (bound at <interactive>:2:5)

We didn't really need a typed hole to tell us this, but at least we know what we have to work with. We know we have to provide a `Cont` value, so let's narrow our typed hole that way.

In [3]:
instance Functor (Cont r) where
    fmap f cont = Cont $ _

<interactive>:2:26: error:
    • Found hole: _ :: (b -> r) -> r
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
             ‘b’ is a rigid type variable bound by
               the type signature for:
                 fmap :: forall a b. (a -> b) -> Cont r a -> Cont r b
               at <interactive>:2:5
    • In the second argument of ‘($)’, namely ‘_’
      In the expression: Cont $ _
      In an equation for ‘fmap’: fmap f cont = Cont $ _
    • Relevant bindings include
        cont :: Cont r a (bound at <interactive>:2:12)
        f :: a -> b (bound at <interactive>:2:10)
        fmap :: (a -> b) -> Cont r a -> Cont r b (bound at <interactive>:2:5)

The type of our hole is more helpful here. Now we know (if we were previously uncertain) that we somehow need to use `f` to turn the `a` into a `b`. We also know that `Cont` takes a parameter, let's add that in and see if it helps.

In [4]:
instance Functor (Cont r) where
    fmap f cont = Cont $ \k -> _

<interactive>:2:32: error:
    • Found hole: _ :: r
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
    • In the expression: _
      In the second argument of ‘($)’, namely ‘\ k -> _’
      In the expression: Cont $ \ k -> _
    • Relevant bindings include
        k :: b -> r (bound at <interactive>:2:27)
        cont :: Cont r a (bound at <interactive>:2:12)
        f :: a -> b (bound at <interactive>:2:10)
        fmap :: (a -> b) -> Cont r a -> Cont r b (bound at <interactive>:2:5)

In general, we know all of our definitions will be of the form `Cont $ \k -> _` and that's a safe starting point. We now know that we need to use `k` on the result of applying `f` to some `a` to finally result in an `r`, but where does the `a` come from? The only thing we can do at this point is 'unwrap' the `cont` using `>>-`. What happens when we do that?

In [5]:
instance Functor (Cont r) where
    fmap f cont = Cont $ \k -> cont >>- _

<interactive>:2:41: error:
    • Found hole: _ :: a -> r
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
             ‘a’ is a rigid type variable bound by
               the type signature for:
                 fmap :: forall a b. (a -> b) -> Cont r a -> Cont r b
               at <interactive>:2:5
    • In the second argument of ‘>>-’, namely ‘_’
      In the expression: cont >>- _
      In the second argument of ‘($)’, namely ‘\ k -> cont >>- _’
    • Relevant bindings include
        k :: b -> r (bound at <interactive>:2:27)
        cont :: Cont r a (bound at <interactive>:2:12)
        f :: a -> b (bound at <interactive>:2:10)
        fmap :: (a -> b) -> Cont r a -> Cont r b (bound at <interactive>:2:5)

It looks like we might have everything we need to complete this definition! We can create a function of type `a -> r` by composing `k` and `f`. 

In [6]:
instance Functor (Cont r) where
    fmap f cont = Cont $ \k -> cont >>- (k . f)

It worked! This definition states that `fmap` works by creating a continuation that expects a callback of the new type. This is pretty exciting! Let's continue to `Applicative`.

In [7]:
instance Applicative (Cont r) where
    pure a = Cont $ \k -> _

<interactive>:2:27: error:
    • Found hole: _ :: r
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
    • In the expression: _
      In the second argument of ‘($)’, namely ‘\ k -> _’
      In the expression: Cont $ \ k -> _
    • Relevant bindings include
        k :: a -> r (bound at <interactive>:2:22)
        a :: a (bound at <interactive>:2:10)
        pure :: a -> Cont r a (bound at <interactive>:2:5)

That was pretty easy. We need an `r` and we have an `a` and a `k` that takes an `a` to an `r`.

In [8]:
instance Applicative (Cont r) where
    pure a = Cont $ \k -> k a

This matches our intuition from above: creating a continuation involves hiding a value behind a function that can access it. On to `<*>`!

In [9]:
instance Applicative (Cont r) where
    pure a  = Cont $ \k -> k a
    f <*> a = Cont $ \k -> _

<interactive>:3:28: error:
    • Found hole: _ :: r
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
    • In the expression: _
      In the second argument of ‘($)’, namely ‘\ k -> _’
      In the expression: Cont $ \ k -> _
    • Relevant bindings include
        k :: b -> r (bound at <interactive>:3:23)
        a :: Cont r a (bound at <interactive>:3:11)
        f :: Cont r (a -> b) (bound at <interactive>:3:5)
        (<*>) :: Cont r (a -> b) -> Cont r a -> Cont r b (bound at <interactive>:3:5)

From above, we know we can 'unwrap' `Cont` values using `>>-`.

In [10]:
instance Applicative (Cont r) where
    pure a  = Cont $ \k -> k a
    f <*> a = Cont $ \k -> f >>- _

<interactive>:3:34: error:
    • Found hole: _ :: (a -> b) -> r
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
             ‘b’ is a rigid type variable bound by
               the type signature for:
                 (<*>) :: forall a b. Cont r (a -> b) -> Cont r a -> Cont r b
               at <interactive>:3:7
             ‘a’ is a rigid type variable bound by
               the type signature for:
                 (<*>) :: forall a b. Cont r (a -> b) -> Cont r a -> Cont r b
               at <interactive>:3:7
    • In the second argument of ‘>>-’, namely ‘_’
      In the expression: f >>- _
      In the second argument of ‘($)’, namely ‘\ k -> f >>- _’
    • Relevant bindings include
        k :: b -> r (bound at <interactive>:3:23)
        a :: Cont r a (bound at <interactive>:3:11)
        f :: Cont r (a -> b) (bound at <interactive>:3:5)
        (<*>) :: Cont r (a -> b) -> Cont r a -> Cont r b (bound at <interactive>:3:5)

Let's keep going.

In [11]:
instance Applicative (Cont r) where
    pure a  = Cont $ \k -> k a
    f <*> a = Cont $ \k -> f >>- \f' -> a >>- \a' -> _

<interactive>:3:54: error:
    • Found hole: _ :: r
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
    • In the expression: _
      In the second argument of ‘>>-’, namely ‘\ a' -> _’
      In the expression: a >>- \ a' -> _
    • Relevant bindings include
        a' :: a (bound at <interactive>:3:48)
        f' :: a -> b (bound at <interactive>:3:35)
        k :: b -> r (bound at <interactive>:3:23)
        a :: Cont r a (bound at <interactive>:3:11)
        f :: Cont r (a -> b) (bound at <interactive>:3:5)
        (<*>) :: Cont r (a -> b) -> Cont r a -> Cont r b (bound at <interactive>:3:5)

Perfect, we want an `r` and we have

- an `a` (`a'`)
- a function from `a` to `b` (`f'`)
- a function from `b` to `r` (`k`)

Let's put them together.

In [12]:
instance Applicative (Cont r) where
    pure a  = Cont $ \k -> k a
    f <*> a = Cont $ \k -> f >>- \f' -> a >>- \a' -> k (f' a')

Okay, we unwrap the function and the argument and rewrap them in a fresh continuation, not too differently from how we defined `fmap`. Sweet! On to the big M!

In [13]:
instance Monad (Cont r) where
    a >>= f = Cont $ \k -> _

<interactive>:2:28: error:
    • Found hole: _ :: r
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
    • In the expression: _
      In the second argument of ‘($)’, namely ‘\ k -> _’
      In the expression: Cont $ \ k -> _
    • Relevant bindings include
        k :: b -> r (bound at <interactive>:2:23)
        f :: a -> Cont r b (bound at <interactive>:2:11)
        a :: Cont r a (bound at <interactive>:2:5)
        (>>=) :: Cont r a -> (a -> Cont r b) -> Cont r b (bound at <interactive>:2:5)

As before, our first order of business is to unwrap the `a`.

In [14]:
instance Monad (Cont r) where
    a >>= f = Cont $ \k -> a >>- \a' -> _

<interactive>:2:41: error:
    • Found hole: _ :: r
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
    • In the expression: _
      In the second argument of ‘>>-’, namely ‘\ a' -> _’
      In the expression: a >>- \ a' -> _
    • Relevant bindings include
        a' :: a (bound at <interactive>:2:35)
        k :: b -> r (bound at <interactive>:2:23)
        f :: a -> Cont r b (bound at <interactive>:2:11)
        a :: Cont r a (bound at <interactive>:2:5)
        (>>=) :: Cont r a -> (a -> Cont r b) -> Cont r b (bound at <interactive>:2:5)

We can apply `f` to this unwrapped value to get a continuation that we can unwrap again.

In [15]:
instance Monad (Cont r) where
    a >>= f = Cont $ \k -> a >>- \a' -> f a' >>- \f' -> _

<interactive>:2:57: error:
    • Found hole: _ :: r
      Where: ‘r’ is a rigid type variable bound by the instance declaration at <interactive>:1:10
    • In the expression: _
      In the second argument of ‘>>-’, namely ‘\ f' -> _’
      In the expression: f a' >>- \ f' -> _
    • Relevant bindings include
        f' :: b (bound at <interactive>:2:51)
        a' :: a (bound at <interactive>:2:35)
        k :: b -> r (bound at <interactive>:2:23)
        f :: a -> Cont r b (bound at <interactive>:2:11)
        a :: Cont r a (bound at <interactive>:2:5)
        (>>=) :: Cont r a -> (a -> Cont r b) -> Cont r b (bound at <interactive>:2:5)

We want an `r` and we have `k` and `f'`. Let's put them together!

In [16]:
instance Monad (Cont r) where
    a >>= f = Cont $ \k -> a >>- \a' -> f a' >>- \f' -> k f'

And that's it! The [mother of all monads](http://blog.sigfpe.com/2008/12/mother-of-all-monads.html) boils down to some tedious and almost mechanical wrapping and unwrapping. I think it's cool how mundane it is.

Still a bit wary? That's fair. I like to poke at the definitions, [read the source](https://hackage.haskell.org/package/transformers-0.4.3.0/docs/src/Control-Monad-Trans-Cont.html), look at how Gabriel Gonzales [explains it](http://www.haskellforall.com/2014/04/how-continuation-monad-works.html), and have a cup of tea and think about life for a while. Whatever works for you!

If you looked at the source, you might have noticed something interesting: The definition for the ContT monad transformer is identical! Here it is below.

In [17]:
newtype ContT r m a = ContT { (>>>-) :: (a -> m r) -> m r }

instance Monad m => Functor (ContT r m) where
    fmap f cont = ContT $ \k -> cont >>>- (k . f)

instance Monad m => Applicative (ContT r m) where
    pure a  = ContT $ \k -> k a
    f <*> a = ContT $ \k -> f >>>- \f' -> a >>>- \a' -> k (f' a')

instance Monad m => Monad (ContT r m) where
    a >>= f = ContT $ \k -> a >>>- \a' -> f a' >>>- \f' -> k f'

I love being able to interact with these definitions like this. This is really how I want to program, and I'd encourage you to try it! The notebook is [here](https://github.com/vaibhavsagar/notebooks/blob/master/notebooks/Continuation.ipynb) for you to play with if you have IHaskell set up. As of this writing, I'm using [this fork](https://github.com/abarbu/IHaskell) but I'm hopeful that changes will be integrated back into [the original](https://github.com/gibiansky/IHaskell) soon.

I feel like I should end with something profound about continuations, but I'll instead link you to [this presentation by Tim Humphries](http://teh.id.au/posts/2017/05/10/lambdajam-slides/index.html) and once again nudge you to try typed holes the next time you're in a Haskell bind (pun very much intended).

Thanks to Iain McCoy, Julia Evans, and Carl Factora for their feedback and suggestions on this post.